<a href="https://colab.research.google.com/github/Thuan20146215/AI/blob/main/FinalPJ_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [22]:
df=pd.read_csv('train.csv')
df=df.dropna()###Drop Nan Values
X=df.drop('label',axis=1)
#X=df['title']
y=df['label']

In [3]:
y.head(10)

0     1
1     0
2     1
3     1
4     1
5     0
7     0
9     0
10    0
11    0
Name: label, dtype: int64

In [4]:
### Vocabulary size
voc_size=5000

In [21]:
messages=X.copy()

messages.reset_index(inplace=True)


In [8]:
import nltk
import re
from nltk.corpus import stopwords

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

In [25]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4623  870 1998]
 [   0    0    0 ...  133 2887 2619]
 [   0    0    0 ... 4189 3200  171]
 ...
 [   0    0    0 ... 3340 1450 3052]
 [   0    0    0 ... 2099  856  772]
 [   0    0    0 ...   53 3720 3061]]


In [26]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1085,
       3937, 2404, 3971, 1265, 4135,  349, 4623,  870, 1998], dtype=int32)

In [33]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_4 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 266,601
Trainable params: 266,601
Non-tr

In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [34]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [30]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

In [37]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64)

Epoch 1/100
200/200 [==============================] - 13s 64ms/step - loss: 0.3025 - accuracy: 0.8746 - val_loss: 0.2059 - val_accuracy: 0.9112
Epoch 2/100
200/200 [==============================] - 11s 55ms/step - loss: 0.1489 - accuracy: 0.9424 - val_loss: 0.2065 - val_accuracy: 0.9127
Epoch 3/100
200/200 [==============================] - 11s 56ms/step - loss: 0.1078 - accuracy: 0.9594 - val_loss: 0.2374 - val_accuracy: 0.9151
Epoch 4/100
200/200 [==============================] - 9s 46ms/step - loss: 0.0751 - accuracy: 0.9737 - val_loss: 0.2626 - val_accuracy: 0.9065
Epoch 5/100
200/200 [==============================] - 11s 57ms/step - loss: 0.0513 - accuracy: 0.9816 - val_loss: 0.3361 - val_accuracy: 0.9107
Epoch 6/100
200/200 [==============================] - 11s 56ms/step - loss: 0.0364 - accuracy: 0.9866 - val_loss: 0.3688 - val_accuracy: 0.9078
Epoch 7/100
200/200 [==============================] - 11s 54ms/step - loss: 0.0257 - accuracy: 0.9914 - val_loss: 0.4123 - val_acc

In [38]:
y_pred=(model.predict(X_test) > 0.5).astype("int32")

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

172/172 [==============================] - 2s 10ms/step


0.9032081662413416

In [39]:
print(y_test[:10])
print(y_pred[:10])

[1 0 0 0 1 1 0 1 1 1]
[[1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]]


In [40]:
##chay thu
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus1 = []
text = input('Enter Title: ')
for i in range(0, len(text)):
    
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus1.append(review)

onehot_repr1=[one_hot(words,voc_size)for words in corpus1] 
embedded_docs1=pad_sequences(onehot_repr1,padding='pre',maxlen=20)
test = np.array(embedded_docs1)
y_pred=(model.predict(test) > 0.5).astype("int32")
if (y_pred[0]==1):
  print('Fake New !!!')
else:
  print('Real New !!!')

Enter Title: Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (Exclusive Video) - Breitbart
4/4 [==============================] - 0s 9ms/step
Real New !!!


In [42]:
from nltk.stem.porter import PorterStemmer
def runtest(text):
  ps = PorterStemmer()
  corpus1 = []
  for i in range(0, len(text)):
     
      review = re.sub('[^a-zA-Z]', ' ', text)
      review = review.lower()
      review = review.split()
      
      review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
      review = ' '.join(review)
      corpus1.append(review)

  onehot_repr1=[one_hot(words,voc_size)for words in corpus1] 
  embedded_docs1=pad_sequences(onehot_repr1,padding='pre',maxlen=20)
  test = np.array(embedded_docs1)
  y_pred=(model.predict(test) > 0.5).astype("int32")
  return y_pred[0]


In [47]:
text = input('Enter Title: ')
x = runtest(text)

if (x==1):
  print(text + ' is a Fake New !!!')
else:
  print(text + ' is a Real New !!!')

Enter Title: Iranian woman jailed for fictional unpublished story about woman stoned to death for 
3/3 [==============================] - 0s 7ms/step
Iranian woman jailed for fictional unpublished story about woman stoned to death for  is a Fake New !!!


In [41]:
model.save("/content/drive/MyDrive/ai/FnPJ3.h5")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')